In [1]:
#necessary imports
import pandas as pd
import numpy as np

from datetime import datetime

In [2]:
#create dataframe to hold final values
pd.set_option('display.max_rows', None)
colnames = ['user', 'date', 'numlogonDay', 'numlogonNight','numlogoffDay','numlogoffNight', 'numPClogonDay', 'numPClogonNight', 'numPClogoffDay', 'numPClogoffNight', 'onoffNotsameDay', 'onoffNotsameNight']
results = pd.DataFrame(columns = colnames)

In [3]:
#file holding separated date file names
file = open('logon_data/logon_files.log', 'r')

#loop through date files
while True:
    line = file.readline().strip()
    
    #if there is no line, leave the loop
    if not line:
        break
    
    #read the needed files
    df = pd.read_csv(f'logon_data/{line}')
    current_date = df.only_date.iloc[0]
    
    #group the data by user
    grouped = df.groupby('user')
    
    #loop through the grouped data
    for user, group in grouped:
        
        #parse needed data
        attributes = {
            'user' : user,
            'date' : current_date,
            'numlogonDay' : len(group.loc[(group.WorkingHours == 'Work Hours') & (group.activity == 'Logon')]),
            'numlogonNight' : len(group.loc[(group.WorkingHours == 'Off Hours') & (group.activity == 'Logon')]),
            'numlogoffDay' : len(group.loc[(group.WorkingHours == 'Work Hours') & (group.activity == 'Logoff')]),
            'numlogoffNight' : len(group.loc[(group.WorkingHours == 'Off Hours') & (group.activity == 'Logoff')]),
            'numPClogonDay' : len(group.loc[(group.WorkingHours == 'Work Hours') & (group.activity == 'Logon')].pc.unique()),
            'numPClogonNight' : len(group.loc[(group.WorkingHours == 'Off Hours') & (group.activity == 'Logon')].pc.unique()),
            'numPClogoffDay' : len(group.loc[(group.WorkingHours == 'Work Hours') & (group.activity == 'Logoff')].pc.unique()),
            'numPClogoffNight' : len(group.loc[(group.WorkingHours == 'Off Hours') & (group.activity == 'Logoff')].pc.unique()),
            'onoffNotsameDay' : len(group.loc[(group.activity == 'Logon') & (group.WorkingHours == 'Work Hours')]) - len(group.loc[(group.activity == 'Logoff') & (group.WorkingHours == 'Work Hours')]), 
            'onoffNotsameNight' : len(group[(group.activity == 'Logon') & (group.WorkingHours == 'Off Hours')]) - len(group[(group.activity == 'Logoff') & (group.WorkingHours == 'Off Hours')]) 
        }
    
        #add the data to the dataframe
        results.loc[len(results)] = [attributes[column] for column in colnames]

In [5]:
#apply abs to the last two columns
results['onoffNotsameDay'] = results['onoffNotsameDay'].abs()
results['onoffNotsameNight'] = results['onoffNotsameNight'].abs()

In [6]:
#store the results
results.to_csv('new_parsed_logon.csv')